# Web Crawler

In [1]:
import requests
from bs4 import BeautifulSoup as soup  # HTML data structure

# extract article hyperlinks from an index page
def extractArtLinks(url):
    r = requests.get(url, cookies={'over18':'1'})
    page_soup = soup(r.text, "html.parser")
    containers = page_soup.findAll("div", {"class": "r-ent"})
    art_links = []
    for container in containers:
        # Finds all link tags "a" from within the first div.
        art_link = container.find('a')
        if art_link:
            #print(art_link['href'])
            #print(container.find('div',{'class':'title'}).get_text())
            art_meta = container.find('div',{'class':'meta'})
            #print(art_meta.find('div',{'class':'author'}).get_text())
            #print(art_meta.find('div',{'class':'date'}).get_text())

            art_links.append({
                'push': container.find('div',{'class':'nrec'}).get_text(),
                'title': container.find('div',{'class':'title'}).get_text().strip(),
                'date': art_meta.find('div',{'class':'date'}).get_text(),
                'author': art_meta.find('div',{'class':'author'}).get_text(),
                'link': art_link['href'],
                'text': extractArtText('https://www.ptt.cc' + art_link['href'])
            })

    return(art_links)

# find the previous index page link
def findPrevIndex(url):
    r = requests.get(url, cookies={'over18':'1'})
    page_soup = soup(r.text,"html.parser")
    btn = page_soup.select('div.btn-group > a')
    up_page_href = btn[3]['href']
    next_page_url = 'https://www.ptt.cc' + up_page_href
    return(next_page_url)

# extract article contents from  the article hyperlink
def extractArtText(url):
    r = requests.get(url, cookies={'over18':'1'})
    page_soup = soup(r.text, "lxml")
    #print(page_soup.find("div",{"id":"main-content"}).get_text())
    art_text=page_soup.select('div#main-content', limit=1)[0].text
    return(art_text)

In [2]:
# main()
num_of_index_page = 2
board_name = 'Food'
url = 'https://www.ptt.cc/bbs/{}/index.html'.format(board_name)
all_links =[]
for page in range(1,num_of_index_page):
    all_links = all_links + extractArtLinks(url)
    url = findPrevIndex(url)
len(all_links)

17

In [3]:
type(all_links[2])
print(all_links[2])

{'push': '', 'title': '[食記] 台北大安美式Diner漢堡店｜Everywhere', 'date': '10/03', 'author': 'TooQoo', 'link': '/bbs/Food/M.1601735629.A.BD6.html', 'text': '作者TooQoo (禿酷)看板Food標題[食記] 台北大安美式Diner漢堡店｜Everywhere 時間Sat Oct  3 22:33:43 2020\n\n   餐廳名稱：Everywhere Burger Club 漢堡俱樂部\n   消費時間：2020/7\n   地址：台北市大安區光復南路420巷21號\n   營業時間：11:30–15:00，17:00–22:00 (週五、六至22:30，週一公休)\n   每人平均價位：250-350\n   推薦菜色：煙燻番茄醬/壓扁雙層牛 老派南方炸雞 紅甜薯片\n   官網：https://www.facebook.com/everywherebc/\n\n----------------------------------\n\n圖文食記：\nhttps://adaitalk.com/taiwanfoodie/taipeifoodie/6813/\n\n----------------------------------\n\nhttps://imgur.com/wsRuFkS\n\n【環境】\n我們大約18:40抵達，入店內登記後位，\n前面約有5組2位的客人，\n等待大約30分鐘後輪到我們!\n(無法留電話等提醒喔!)\n下圖照片是20:15左右離開時的狀況\n\n很喜歡霓虹燈管的風格呀!\n\n\n只有一個漢堡圖案的燈牌，很有趣耶!\n\n\n雖然我們近期都愛去老屋改造的店，\n但是這間的風格強烈，且很有整體性，\n很有誠意的裝潢~ 我們很愛!\n\n一進門會聞到蠻濃的料理味，肚子會超餓~\n不知道坐吧台吃完後身上會不會很有味道?!\n(我們剛好被排到旁邊的座位區)\n\n\n【餐點】\n菜單\n\n\n煙燻番茄醬/壓扁雙層牛 + 經典薯條/可樂\nNTD 215 + 70\n新鮮牛番茄/自製煙燻番茄醬/牛肉排x2/切達起司x2/焦香洋蔥圈\n漢堡 麵包並不是烤到脆的類型，\n是很又鬆又厚又有彈性的布里歐麵包 

In [4]:
print('Push: {push:s} \n'
      'title: {title:s} \n'
      'date: {date:s} \n'
      'author: {author:s} \n'
      'link: {link:s} \n'
      'text: {text:.5} \n'.format(**all_links[2]))

Push:  
title: [食記] 台北大安美式Diner漢堡店｜Everywhere 
date: 10/03 
author: TooQoo 
link: /bbs/Food/M.1601735629.A.BD6.html 
text: 作者Too 



:::{admonition} Exercise
How to seperate post texts from push texts?
:::
